In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datasets
import pandas as pd
from tqdm.auto import tqdm
from itertools import chain

In [3]:
#ri = datasets.ReadInstruction("train") + datasets.ReadInstruction("test")
dt = datasets.load_dataset("/storage/shared_data/manu/zinc_dump/", streaming=False, num_proc=32, cache_dir="/storage/shared_data/manu/.cache")

Resolving data files:   0%|          | 0/127 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/66 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/66 [00:00<?, ?it/s]

Found cached dataset arrow (/storage/shared_data/manu/.cache/arrow/zinc_dump-c4ef570674f58a63/0.0.0/74f69db2c14c2860059d39860b1f400a03d11bf7fb5a8258ca38c501c878c137)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dt = dt.cast_column("id", datasets.Value("string"))

Loading cached processed dataset at /storage/shared_data/manu/.cache/arrow/zinc_dump-c4ef570674f58a63/0.0.0/74f69db2c14c2860059d39860b1f400a03d11bf7fb5a8258ca38c501c878c137/cache-28a161d2699fd303.arrow
Loading cached processed dataset at /storage/shared_data/manu/.cache/arrow/zinc_dump-c4ef570674f58a63/0.0.0/74f69db2c14c2860059d39860b1f400a03d11bf7fb5a8258ca38c501c878c137/cache-dc4d6324982b4506.arrow
Loading cached processed dataset at /storage/shared_data/manu/.cache/arrow/zinc_dump-c4ef570674f58a63/0.0.0/74f69db2c14c2860059d39860b1f400a03d11bf7fb5a8258ca38c501c878c137/cache-4a4b63c77f3cba59.arrow


In [5]:
def update_info(examples):
    return {"source": ["zinc-20"]*len(examples["id"])}
updated_dt =  dt.map(update_info, num_proc=32, batched=True, batch_size=5000)

Loading cached processed dataset at /storage/shared_data/manu/.cache/arrow/zinc_dump-c4ef570674f58a63/0.0.0/74f69db2c14c2860059d39860b1f400a03d11bf7fb5a8258ca38c501c878c137/cache-e309a2078735483d_*_of_00032.arrow
Loading cached processed dataset at /storage/shared_data/manu/.cache/arrow/zinc_dump-c4ef570674f58a63/0.0.0/74f69db2c14c2860059d39860b1f400a03d11bf7fb5a8258ca38c501c878c137/cache-fcb8d27b42171c00_*_of_00032.arrow
Loading cached processed dataset at /storage/shared_data/manu/.cache/arrow/zinc_dump-c4ef570674f58a63/0.0.0/74f69db2c14c2860059d39860b1f400a03d11bf7fb5a8258ca38c501c878c137/cache-dc8e5007ce21b30e_*_of_00032.arrow


In [6]:
unichem_df = pd.read_parquet("/storage/shared_data/manu/unichem", engine="fastparquet")

In [7]:
unichem_df["id"] = unichem_df["id"].astype("str")

In [8]:
unichem_dt_tmp = datasets.Dataset.from_pandas(unichem_df.drop(columns=["parquet_partition"]))

In [9]:
# 80% train, 20% test + validation
train_test_valid = unichem_dt_tmp.train_test_split(test_size=0.2)
# split the 20 % test into  test and validation
test_valid = train_test_valid["test"].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
unichem_dt = datasets.DatasetDict({
    "train": train_test_valid["train"],
    "test": test_valid["test"],
    "validation": test_valid["train"]})

In [10]:
unichem_dt = unichem_dt.select_columns(['id', 'smiles', 'source'])

In [11]:
test_dt = datasets.concatenate_datasets([unichem_dt["test"], updated_dt["test"]])

In [12]:
validation_dt = datasets.concatenate_datasets([unichem_dt["validation"], dt["validation"]])

In [14]:
train_dt = datasets.concatenate_datasets([unichem_dt["train"], dt["train"]])

In [15]:
dataset = datasets.DatasetDict(dict(train=train_dt, test=test_dt, validation=validation_dt))

In [ ]:
dataset.save_to_disk("/storage/shared_data/manu/processed_zinc_unichem", max_shard_size="2GB", num_proc=8)

Saving the dataset (0/39 shards):   0%|          | 0/947392825 [00:00<?, ? examples/s]